# UCI Adult Dataset or Census Income

This is a very popular ML task, with tabular data. The objective is to predict whether income exceeds $50K/yr based on census data. 
Also known as "Census Income" dataset.

The data is old and biased on different ways ... but it can be used opaquely for ML experimentation.



## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

GoMLX provides [a simple `adult` library](https://pkg.go.dev/github.com/gomlx/gomlx/examples/adult) to facilitate downdoaling and preprocessing the data. Data is available in [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).

After downloading the data and validating the checksum (both training and testing), it generates the quantiles for the continuous features, and the vocabularies for the categorical features. It saves all this info for faster restart later in a binary file. So this won't be necessary a second time.

The quantiles are used to calibrate the values, using a piece-wise-lienar calibration, very good for these things. See [`layers.PieceWiseLinearCalibration` documentation](https://pkg.go.dev/github.com/gomlx/gomlx@v0.1.0/ml/layers#PieceWiseLinearCalibration).

We create a flag `--data` to define the directory where to save the intermediary files: downloaded and preprocessed datasets.
In this examle we set it to `~/work/uci-adult`. Verbosity can be contolled with the `--verbosity` flag. 

We set default in Go for these flags, but they can easily be reset for a new run by providing them after the `%%` Jupyter kernel meta-command -- in indicates that the subsequent lines should be put in to a `func main`.


In [2]:
import (
    "flag"
    
    "github.com/gomlx/gomlx/examples/adult"
)

var (
    flagDataDir       = flag.String("data", "~/work/uci-adult", "Directory to save and load downloaded and generated dataset files.")
    flagVerbosity     = flag.Int("verbosity", 0, "Level of verbosity, the higher the more verbose.")
    flagForceDownload = flag.Bool("force_download", false, "Force re-download of Adult dataset files.")
    flagNumQuantiles  = flag.Int("quantiles", 100, "Max number of quantiles to use for numeric features, used during piece-wise linear calibration. It will only use unique values, so if there are fewer variability, fewer quantiles are used.")
)

%% --verbosity=2
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)


Sample Categorical: (24.08% positive ratio, 23.86% weighted positive ratio)
	Row 0:	[7 10 5 1 2 5 2 39]
	Row 1:	[6 10 3 4 1 5 2 39]
	Row 2:	[4 12 1 6 2 5 2 39]
	...
	Row 32558:	[4 12 7 1 5 5 1 39]
	Row 32559:	[4 12 5 1 4 5 2 39]
	Row 32560:	[5 12 3 4 6 5 1 39]

Sample Continuous:
	Row 0:	[39 13 2174 0 40]
	Row 1:	[50 13 0 0 13]
	Row 2:	[38 9 0 0 40]
	...
	Row 32558:	[58 9 0 0 40]
	Row 32559:	[22 9 0 0 20]
	Row 32560:	[52 9 15024 0 40]


In [3]:
!ls -lh ~/work/uci-adult

total 7.0M
-rw-r--r-- 1 janpf janpf 3.8M Mar 21 09:14 adult.data
-rw-r--r-- 1 janpf janpf 1.3M Mar 21 09:14 adult_data-100_quantiles.bin
-rw-r--r-- 1 janpf janpf 2.0M Mar 21 09:14 adult.test


### Creating Datasets

First we create the GoMLX's `Manager`: it's the object that manages the underlying XLA
setup, connection and execution. It's needed to create tensors.

With that we create the samplers of data that we will use to train and evaluate. They implement 
GoMLX's `train.Dataset` interface, which is what is used by our training loop to draw batches to
train, or our eval loop to draw batches to evaluate.

The inputs are 3 tensors: *categorical values*, *continuous values* and *weights*.

In the cell below we define the `Manager` flags, `BuildSamplers` and printout some samples.

In [4]:
import (
    "flag"
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/tensor"
)

var (
    flagBatchSize      = flag.Int("batch", 128, "BatchSampler size for training")
)

// Global manager created an initialization, used everywhere.
var manager = NewManager()

// BuildDatasets returns 3 `train.Dataset`:
// * trainingSampler is an endless random sampler used for training.
// * trainingEvalSampler samples through exactly one epoch of the train dataset.
// * testEvalSampler samples through exactly one epoch of the test dataset.
func BuildDatasets(manager *Manager) (trainDS, trainEvalDS, testEvalDS train.Dataset) {
    baseDS := adult.NewDataset(manager, adult.Data.Train, "batched train")
    trainEvalDS = baseDS.Copy().BatchSize(*flagBatchSize, false)
    testEvalDS = adult.NewDataset(manager, adult.Data.Test, "test").
        BatchSize(*flagBatchSize, false)
    // For training, we shuffle and loop indefinitely.
    trainDS = baseDS.BatchSize(*flagBatchSize, true).Shuffle().Infinite(true)
    return
}

// PositiveRatio finds out the the ratio of positive labels in the
// training and testing data.
//
// We could do this easily with GoMLX computation model (just `ReduceAllSum`), but
// this examples shows it's also ok to mix Go computations.
func PositiveRatio(ds train.Dataset) float32 {
    ds.Reset()  // Start from beginning.
    var sum float32
    var count float32
    for {
        _, _, labels, err := ds.Yield()
        if err == io.EOF {
            break;
        }
        if err != nil { panic(err) }
        data := labels[0].Local().Flat().([]float32)
        for _, value := range data {
            sum += value
        }
        count += float32(len(data))
    }
    return sum/count
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
trainingDS, trainingEvalDS, testEvalDS := BuildDatasets(manager)

// Take one batch.
_, inputs, labels, err := trainingDS.Yield()
if err != nil { panic(err) }
fmt.Printf("Inputs of batch (size %d):\n", *flagBatchSize)
fmt.Printf("\tcategorical:\n\t\tFeatures=%v\n", adult.Data.VocabulariesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[0].Local().StringN(16))
fmt.Printf("\tcontinuous:\n\t\tFeatures=%v\n", adult.Data.QuantilesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[1].Local().StringN(10))
fmt.Printf("\tweights: %s\n", inputs[2].Local().StringN(5))
fmt.Printf("\nLabels of batch:\n\t%s\n", labels[0].Local().StringN(10))
fmt.Printf("\nLabels distributions:\n\tTrain:\t%.2f%% positive\n\tTest:\t%.2f%% positive\n",
           PositiveRatio(trainingEvalDS)*100.0, PositiveRatio(testEvalDS)*100.0)


Inputs of batch (size 128):
	categorical:
		Features=[workclass education marital-status occupation relationship race sex native-country]
		Values: (Int64)[128 8]: (... too large, 1024 values ..., first 16 values: [7 1 3 14 1 5 2 39 4 16 5 13 2 5 1 39])
	continuous:
		Features=[age education-num capital-gain capital-loss hours-per-week]
		Values: (Float32)[128 5]: (... too large, 640 values ..., first 10 values: [53 6 0 0 40 27 10 0 0 30])
	weights: (Float32)[128 1]: (... too large, 128 values ..., first 5 values: [153486 188189 287927 407684 137088])

Labels of batch:
	(Float32)[128 1]: (... too large, 128 values ..., first 10 values: [1 0 1 0 0 0 0 0 0 1])

Labels distributions:
	Train:	24.08% positive
	Test:	23.62% positive


## Model Definition

Lots of hyper-parameter flags, but otherwise a straight forward FNN, using piece-wise linear calibration of the continuous features, and embeddings for the categorical features.

> **Note**: building models is a constant checking that shapes are compatible. It's a bit annoying, in particular because shapes are known in runtime only -- no compile time check. GoMLX tries to help providing a stack trace of where errors happen so one can pin-point issues quickly. But often it involves lots of experimentation (more than ordinary Go code).
>
> Developing with a Noteboook (see [GoNB](https://github.com/janpfeifer/gonb)) or simply a unit test on your `ModelGraph` function are quick/convenient ways to develop models -- before actually training them. You can also use shape asserts in the middle of the 
>`ModelGraph`, as we do below.

In [5]:
import (
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/ml/train/optimizers"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    // ModelDType used for the model. Must match RawData Go types.
    ModelDType = shapes.Float32
    

    // Model hyperparameters.
    flagUseCategorical       = flag.Bool("use_categorical", true, "Use categorical features.")
    flagUseContinuous        = flag.Bool("use_continuous", true, "Use continuous features.")
    flagTrainableCalibration = flag.Bool("trainable_calibration", true, "Allow piece-wise linear calibration to adjust outputs.")
    flagEmbeddingDim    = flag.Int("embedding_dim", 8, "Default embedding dimension for categorical values.")
    flagNumHiddenLayers = flag.Int("hidden_layers", 8, "Number of hidden layers, stacked with residual connection.")
    flagNumNodes        = flag.Int("num_nodes", 32, "Number of nodes in hidden layers.")
    flagDropoutRate     = flag.Float64("dropout", 0, "Dropout rate")
    
    // Training parameter, referenced here.
    flagLearningRate  = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagNumSteps      = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
)


// ModelGraph outputs the logits (not the probabilities). The parameter inputs should contain 3 tensors:
//
// - categorical inputs, shaped  `(int64)[batch_size, len(VocabulariesFeatures)]`
// - continuous inputs, shaped `(float32)[batch_size, len(Quantiles)]`
// - weights: not currently used, but shaped `(float32)[batch_size, 1]`.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used, since the dataset is always the same.
    g := inputs[0].Graph()
    
    // Use Cosine schedule of the learning rate.
    optimizers.CosineAnnealingSchedule(ctx, g, ModelDType).
        PeriodInSteps(*flagNumSteps/3).Done()
    
    categorical, continuous := inputs[0], inputs[1]
    batchSize := categorical.Shape().Dimensions[0]
    
    var allEmbeddings []*Node

    if *flagUseCategorical {
        // Embedding of categorical values, each with its own vocabulary.
        numCategorical := categorical.Shape().Dimensions[1]
        for catIdx := 0; catIdx < numCategorical; catIdx++ {
            // Take one column at a time of the categorical values.
            split := Slice(categorical, AxisRange(), AxisRange(catIdx, catIdx+1))
            // Embed it accordingly.
            embedCtx := ctx.In(fmt.Sprintf("categorical_%d_%s", catIdx, adult.Data.VocabulariesFeatures[catIdx]))
            vocab := adult.Data.Vocabularies[catIdx]
            vocabSize := len(vocab)
            embedding := layers.Embedding(embedCtx, split, ModelDType, vocabSize, *flagEmbeddingDim)
            embedding.AssertDims(batchSize, *flagEmbeddingDim) // 2-dim tensor, with batch size as the leading dimension.
            allEmbeddings = append(allEmbeddings, embedding)
        }
    }

    if *flagUseContinuous {
        // Piecewise-linear calibration of the continuous values. Each feature has its own number of quantiles.
        numContinuous := continuous.Shape().Dimensions[1]
        for contIdx := 0; contIdx < numContinuous; contIdx++ {
            // Take one column at a time of the continuous values.
            split := Slice(continuous, AxisRange(), AxisRange(contIdx, contIdx+1))
            featureName := adult.Data.QuantilesFeatures[contIdx]
            calibrationCtx := ctx.In(fmt.Sprintf("continuous_%d_%s", contIdx, featureName))
            quantiles := adult.Data.Quantiles[contIdx]
            layers.AssertQuantilesForPWLCalibrationValid(quantiles)
            calibrated := layers.PieceWiseLinearCalibration(calibrationCtx, split, Const(g, quantiles), *flagTrainableCalibration)
            calibrated.AssertDims(batchSize, 1) // 2-dim tensor, with batch size as the leading dimension.
            allEmbeddings = append(allEmbeddings, calibrated)
        }
    }
    layer := Concatenate(allEmbeddings, -1)
    layer.AssertDims(batchSize, -1) // 2-dim tensor, with batch size as the leading dimension (-1 means it is not checked).
    
    layer = layers.DenseWithBias(ctx.In(fmt.Sprintf("DenseLayer_%d", 0)), layer, *flagNumNodes)
    for ii := 1; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("DenseLayer_%d", ii))
        // Add layer with residual connection.
        tmp := Sigmoid(layer)
        if *flagDropoutRate > 0 {
            tmp = layers.Dropout(ctx, tmp, Scalar(g, ModelDType, *flagDropoutRate))
        }
        tmp = layers.DenseWithBias(ctx, tmp, *flagNumNodes)
        layer = Add(layer, tmp)  // Residual connections
    }
    layer = Sigmoid(layer)
    logits := layers.DenseWithBias(ctx.In("DenseFinal"), layer, 1)
    logits.AssertDims(batchSize, 1) // 2-dim tensor, with batch size as the leading dimension.
    return []*Node{logits}
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    

// Let's just check that we get the right shape from the model function, wihtout any real data.
graph := manager.NewGraph("test")
ctx := context.NewContext(manager)
ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

// Create placeholder (parameters) graph nodes, just to test the graph building is working.
inputs := []*Node{
    // Categorical: shaped [batch_size, num_categorical]
    graph.Parameter("categorical", shapes.Make(shapes.Int64, *flagBatchSize, len(adult.Data.VocabulariesFeatures))),
    // Continuous: shaped [batch_size, num_continuos]
    graph.Parameter("continuous", shapes.Make(shapes.Float32, *flagBatchSize, len(adult.Data.QuantilesFeatures))),
    // Weights: shaped [batch_size, 1]
    graph.Parameter("weights", shapes.Make(shapes.Float32, *flagBatchSize, 1)),    
}
logits := ModelGraph(ctx, nil, inputs)
fmt.Printf("Logits shape for batch_size=%d: %s\n", *flagBatchSize, logits[0].Shape())

Logits shape for batch_size=128: (Float32)[128 1]


## Training Loop

We can create a training loop with only a `Manager`, a `Context` (for the model varibles) and the `ModelGraph` function.

To make it more interesting we also add the following:

* Accuracy metrics for training and testing.
* Checkpoints -- so trained model can be saved, and reloaded.
* A progress-bar that also shows training metrics.
* We dynamically plot how the loss and accuracy evolve.

First we define the corresponding flags and the `trainModel` function, and run it for very few steps to make sure
it is working.

In [6]:
import (
    "fmt"
    "io"
    "time"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/examples/notebook/gonb/margaid"
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/shapes"
    "github.com/gomlx/gomlx/types/slices"
    "github.com/gomlx/gomlx/types/tensor"
    "github.com/janpfeifer/gonb/gonbui"
)

var (
    flagOptimizer      = flag.String("optimizer", "adam", "Type of optimizer to use: 'sgd' or 'adam'")
    flagLearningRate   = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagCheckpoint     = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")
    flagPlots          = flag.Bool("plots", true, "Plots during training: perform periodic evaluations, "+
                                   "save results if --checkpoint is set and draw plots, if in a Jupyter notebook.")
)

func trainModel() {
    // Fixes directories.
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    *flagCheckpoint = data.ReplaceTildeInDir(*flagCheckpoint)
    if *flagCheckpoint != "" && !path.IsAbs(*flagCheckpoint) {
        *flagCheckpoint = path.Join(*flagDataDir, *flagCheckpoint)
    }

    // Load data and create datasets.
    adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
    trainDS, trainEvalDS, testEvalDS := BuildDatasets(manager)

    // Context holds the variables and optionally hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(*flagCheckpoint).Keep(*flagCheckpointKeep).Done()
        if err != nil { panic(err) }
        fmt.Printf("Checkpointing model to %q\n", checkpoint.Dir())
        globalStep := optimizers.GetGlobalStepVar(ctx).Value().Value().(int)
        if globalStep != 0 {
            fmt.Printf("Restarting training from global_step=%d\n", globalStep)
        }
    }

    // Pick a known optimizer.
    optimizerFn, found := optimizers.KnownOptimizers[*flagOptimizer]
    if !found {
        log.Fatalf("Unknown optimizer %q, please use one of %v",
            *flagOptimizer, slices.Keys(optimizers.KnownOptimizers))
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    trainer := train.NewTrainer(manager, ctx, ModelGraph, losses.BinaryCrossentropyLogits,
        optimizerFn(),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    loop.ReadGlobalStep(ctx)  // Make sure it restarts from previous global step, if one is set.
    commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Attach a checkpoint: checkpoint every 1 minute of training.
    if checkpoint != nil {
        period := time.Minute * 1
        train.PeriodicCallback(loop, period, true, "saving checkpoint", 100,
            func(loop *train.Loop, metrics []tensor.Tensor) error {
                fmt.Printf("\n[saving checkpoint@%d] [median train step (ms): %d]\n", loop.LoopStep, loop.MedianTrainStepDuration().Milliseconds())
                return checkpoint.Save()
            })
    }

    // Attach a margaid plots: plot points at exponential steps.
    // Also points are saved, so training (with plots) can be restarted if --checkpoint is set.
    var plots *margaid.Plots
    if *flagPlots {
        plots = margaid.New(1024, 400, trainEvalDS, testEvalDS).LogScaleX()
        if checkpoint != nil {
            // Save plot points.
            _, err := plots.WithFile(path.Join(checkpoint.Dir(), "training_plot_points.json"))
            if err != nil { panic(err) }
        }
        plots.DynamicUpdates()

        // Only plot if (1) it's running in a notebook or if (B) it has a checkpoint directory, where those plot points
        // will be saved.
        if checkpoint != nil || gonbui.IsNotebook {
            // Register plot points at exponential steps.
            train.ExponentialCallback(loop, 100, 1.1, true,
                "Monitor", 0, func(loop *train.Loop, metrics []tensor.Tensor) error {
                    // Update plots with metrics.
                    return plots.AddTrainAndEvalMetrics(loop, metrics)
                })            
        }
    }

    // Run the given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    if err != nil { panic(err) }
    fmt.Printf("\t[Step %d] median train step: %d microseconds\n", loop.LoopStep, loop.MedianTrainStepDuration().Microseconds())

    // Print a final evaluation on train and test datasets.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    if err != nil { panic(err) }
    fmt.Println()
}

// Notice command line flags are passed in the %% notebook command. We set --plot=false here to disable plotting
// since this is only a quick test that our train() loop is working. See below the final run for a full training.
%% --steps=500 --plots=false
trainModel()

Training (500 steps):  100% [========================================] (164 steps/s) [loss=0.326] [~loss=0.384] [~acc=82.62%]        
	[Step 500] median train step: 979 microseconds

Results on batched train:
	Mean Loss (#loss): 0.353
	Mean Accuracy (#acc): 84.21%
Results on test:
	Mean Loss (#loss): 0.350
	Mean Accuracy (#acc): 84.59%



## Final run

With everything working, we can do our final run.

> **Note** here is where someone might want to hyperparameter tune, trying out different hyperparameters.

In [7]:
%% --steps=5000 --checkpoint base_model
trainModel()

Checkpointing model to "/home/janpf/work/uci-adult/base_model"


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.358092e+01 L1.786294e+02,1.834770e+02 L2.773355e+02,1.892240e+02 L3.506075e+02,1.900605e+02 L4.099543e+02,2.082954e+02 L4.607660e+02,2.201249e+02 L5.055983e+02,2.165440e+02 L5.461769e+02,2.294807e+02 L5.835664e+02,2.349524e+02 L6.184621e+02,2.291309e+02 L6.513380e+02,2.394209e+02 L6.825291e+02,2.448016e+02 L7.123676e+02,2.491282e+02 L7.410809e+02,2.511432e+02 L7.688274e+02,2.512008e+02 L7.957190e+02,2.514884e+02 L8.218897e+02,2.535006e+02 L8.474287e+02,2.533282e+02 L8.663200e+02,2.467644e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.342055e+01 L1.786294e+02,1.867816e+02 L2.773355e+02,1.965673e+02 L3.506075e+02,1.979944e+02 L4.099543e+02,2.113727e+02 L4.607660e+02,2.223015e+02 L5.055983e+02,2.193708e+02 L5.461769e+02,2.301299e+02 L5.835664e+02,2.374569e+02 L6.184621e+02,2.230041e+02 L6.513380e+02,2.381531e+02 L6.825291e+02,2.416291e+02 L7.123676e+02,2.459061e+02 L7.410809e+02,2.444047e+02 L7.688274e+02,2.457908e+02 L7.957190e+02,2.491353e+02 L8.218897e+02,2.471755e+02 L8.474287e+02,2.485592e+02 L8.663200e+02,2.419764e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L1.786294e+02,5.704861e+01 L2.773355e+02,1.561048e+02 L3.506075e+02,1.840368e+02 L4.099543e+02,2.004781e+02 L4.607660e+02,2.064867e+02 L5.055983e+02,2.100995e+02 L5.461769e+02,2.180968e+02 L5.835664e+02,2.287239e+02 L6.184621e+02,2.341960e+02 L6.513380e+02,2.400119e+02 L6.825291e+02,2.419244e+02 L7.123676e+02,2.522809e+02 L7.410809e+02,2.511078e+02 L7.688274e+02,2.506114e+02 L7.957190e+02,2.548000e+02 L8.218897e+02,2.513962e+02 L8.474287e+02,2.507940e+02 L8.663200e+02,2.513445e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.680453e+02,0 L1.680453e+02,-6 M2.560033e+02,0 L2.560033e+02,-6 M3.184106e+02,0 L3.184106e+02,-6 M3.668174e+02,0 L3.668174e+02,-6 M4.063686e+02,0 L4.063686e+02,-6 M4.398087e+02,0 L4.398087e+02,-6 M4.687758e+02,0 L4.687758e+02,-6 M4.943267e+02,0 L4.943267e+02,-6 M5.171826e+02,0 L5.171826e+02,-6 M5.378584e+02,0 L5.378584e+02,-6 M6.675479e+02,0 L6.675479e+02,-6 M7.555060e+02,0 L7.555060e+02,-6 M8.179132e+02,0 L8.179132e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 Steps accuracy accuracy metrics Eval on batched train: Mean Accuracy Eval on test: Mean Accuracy Train: Moving Average Accuracy 
 <path d="M1.768000e+01,2.408970e+02 L1.786294e+02,1.201857e+02 L2.773355e+02,9.811865e+01 L3.506075e+02,9.739397e+01 L4.099543e+02,7.294547e+01 L4.607660e+02,6.345511e+01 L5.055983e+02,6.103595e+01 L5.461769e+02,4.973975e+01 L5.835664e+02,3.913330e+01 L6.184621e+02,3.869877e+01 L6.513380e+02,3.334830e+01 L6.825291e+02,2.383558e+01 L7.123676e+02,1.609497e+01 L7.410809e+02,1.261026e+01 L7.688274e+02,9.979423e+00 L7.957190e+02,8.830906e+00 L8.218897e+02,7.806328e+00 L8.474287e+02,5.200000e+00 L8.663200e+02,1.321204e+01 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.379773e+02 L1.786294e+02,1.162952e+02 L2.773355e+02,9.522374e+01 L3.506075e+02,9.443979e+01 L4.099543e+02,7.244451e+01 L4.607660e+02,6.317149e+01 L5.055983e+02,6.117898e+01 L5.461769e+02,5.136407e+01 L5.835664e+02,4.172203e+01 L6.184621e+02,4.239250e+01 L6.513380e+02,3.762930e+01 L6.825291e+02,2.841665e+01 L7.123676e+02,2.227647e+01 L7.410809e+02,1.929320e+01 L7.688274e+02,1.686873e+01 L7.957190e+02,1.607728e+01 L8.218897e+02,1.613559e+01 L8.474287e+02,1.336792e+01 L8.663200e+02,2.185947e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.786294e+02,2.051004e+02 L2.773355e+02,1.355381e+02 L3.506075e+02,1.056093e+02 L4.099543e+02,8.927128e+01 L4.607660e+02,7.141533e+01 L5.055983e+02,6.557491e+01 L5.461769e+02,5.718198e+01 L5.835664e+02,4.385997e+01 L6.184621e+02,4.004246e+01 L6.513380e+02,3.026534e+01 L6.825291e+02,2.716424e+01 L7.123676e+02,1.665257e+01 L7.410809e+02,1.489876e+01 L7.688274e+02,1.429090e+01 L7.9571

Training (5000 steps):  100% [========================================] (490 steps/s) [loss=0.290] [~loss=0.275] [~acc=87.27%]        

[saving checkpoint@5000] [median train step (ms): 0]
	[Step 5000] median train step: 926 microseconds

Results on batched train:
	Mean Loss (#loss): 0.281
	Mean Accuracy (#acc): 87.02%
Results on test:
	Mean Loss (#loss): 0.288
	Mean Accuracy (#acc): 86.77%



Since the model training went well, and it doesn't seem to be yet terribly overfiting, 
let's train further, another 15k steps, for 20K steps in total.		mg.WithRange()


In [8]:
%% --steps=15000 --checkpoint base_model
trainModel()

Checkpointing model to "/home/janpf/work/uci-adult/base_model"
Restarting training from global_step=5000


<path d="M1.768000e+01,2.313992e+01 L1.365173e+02,1.791997e+02 L2.093971e+02,1.848089e+02 L2.634977e+02,1.856253e+02 L3.073165e+02,2.034227e+02 L3.448334e+02,2.149684e+02 L3.779355e+02,2.114733e+02 L4.078967e+02,2.240997e+02 L4.355033e+02,2.294401e+02 L4.612687e+02,2.237583e+02 L4.855427e+02,2.338014e+02 L5.085726e+02,2.390530e+02 L5.306039e+02,2.432758e+02 L5.518045e+02,2.452424e+02 L5.722912e+02,2.452987e+02 L5.921466e+02,2.455794e+02 L6.114698e+02,2.475433e+02 L6.303266e+02,2.473751e+02 L6.442750e+02,2.409687e+02 L6.487916e+02,2.477676e+02 L6.669114e+02,2.487766e+02 L6.847383e+02,2.487205e+02 L7.023018e+02,2.474311e+02 L7.196363e+02,2.452424e+02 L7.367580e+02,2.472629e+02 L7.536877e+02,2.492248e+02 L7.704470e+02,2.491127e+02 L7.870558e+02,2.465336e+02 L8.035318e+02,2.482721e+02 L8.198887e+02,2.485524e+02 L8.361372e+02,2.481039e+02 L8.522929e+02,2.457478e+02 L8.663200e+02,2.470946e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.274347e+01 L1.365173e+02,1.824250e+02 L2.093971e+02,1.919759e+02 L2.634977e+02,1.933688e+02 L3.073165e+02,2.064261e+02 L3.448334e+02,2.170927e+02 L3.779355e+02,2.142323e+02 L4.078967e+02,2.247333e+02 L4.355033e+02,2.318846e+02 L4.612687e+02,2.177784e+02 L4.855427e+02,2.325640e+02 L5.085726e+02,2.359566e+02 L5.306039e+02,2.401310e+02 L5.518045e+02,2.386656e+02 L5.722912e+02,2.400184e+02 L5.921466e+02,2.432827e+02 L6.114698e+02,2.413699e+02 L6.303266e+02,2.427204e+02 L6.442750e+02,2.362955e+02 L6.487916e+02,2.430579e+02 L6.669114e+02,2.439573e+02 L6.847383e+02,2.429454e+02 L7.023018e+02,2.409195e+02 L7.196363e+02,2.411447e+02 L7.367580e+02,2.449687e+02 L7.536877e+02,2.445193e+02 L7.704470e+02,2.439573e+02 L7.870558e+02,2.463161e+02 L8.035318e+02,2.449687e+02 L8.198887e+02,2.450810e+02 L8.361372e+02,2.455302e+02 L8.522929e+02,2.436201e+02 L8.663200e+02,2.441821e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L1.365173e+02,5.580463e+01 L2.093971e+02,1.524842e+02 L2.634977e+02,1.797461e+02 L3.073165e+02,1.957929e+02 L3.448334e+02,2.016574e+02 L3.779355e+02,2.051834e+02 L4.078967e+02,2.129889e+02 L4.355033e+02,2.233610e+02 L4.612687e+02,2.287018e+02 L4.855427e+02,2.343782e+02 L5.085726e+02,2.362448e+02 L5.306039e+02,2.463529e+02 L5.518045e+02,2.452079e+02 L5.722912e+02,2.447234e+02 L5.921466e+02,2.488115e+02 L6.114698e+02,2.454894e+02 L6.303266e+02,2.449016e+02 L6.442750e+02,2.454389e+02 L6.487916e+02,2.416603e+02 L6.669114e+02,2.496888e+02 L6.847383e+02,2.471988e+02 L7.023018e+02,2.513812e+02 L7.196363e+02,2.471135e+02 L7.367580e+02,2.480603e+02 L7.536877e+02,2.502084e+02 L7.704470e+02,2.437076e+02 L7.870558e+02,2.458886e+02 L8.035318e+02,2.548000e+02 L8.198887e+02,2.530125e+02 L8.361372e+02,2.451103e+02 L8.522929e+02,2.474924e+02 L8.663200e+02,2.537227e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps accuracy accuracy metrics Eval on batched train: Mean Accuracy Eval on test: Mean Accuracy Train: Moving Average Accuracy 
 <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.416988e+02 L1.365173e+02,1.279491e+02 L2.093971e+02,1.071546e+02 L2.634977e+02,1.064717e+02 L3.073165e+02,8.343319e+01 L3.448334e+02,7.449014e+01 L3.77935

Training (15000 steps):  100% [========================================] (766 steps/s) [loss=0.295] [~loss=0.263] [~acc=87.72%]        

[saving checkpoint@20000] [median train step (ms): 0]
	[Step 20000] median train step: 933 microseconds

Results on batched train:
	Mean Loss (#loss): 0.269
	Mean Accuracy (#acc): 87.36%
Results on test:
	Mean Loss (#loss): 0.281
	Mean Accuracy (#acc): 87.20%

